# Implementing a Finite Algebra in Python

In the discussion and code to follow, **ALL** elements of an algebra will be implemented as Python strings, usually lowercase. We could implement elements as classes/objects themselves, but that would be inefficient. Recall, that ALL of a finite algebra's properties can be derived from its Cayley table, so we will focus on creating an efficient Cayley table implementation.

We will start the implementation by working with a simple finite algebra, Rock-Paper-Scissors.

Here are its elements, as a list of strings, and its Cayley table, as a list of lists of strings.

In [1]:
>>> rps_elements = ['r', 'p', 's']

In [2]:
>>> rps_table =[['r', 'p', 'r'],
>>>             ['p', 'p', 's'],
>>>             ['r', 's', 's']]

Using the *list* method, *index*, we can obtain the 0-based index of an element in the list of elements. For RPS, an index will be one of 0, 1, or 2.

Here are some examples:

In [3]:
>>> print(f"{rps_elements.index('r') = }")
>>> print(f"{rps_elements.index('p') = }")
>>> print(f"{rps_elements.index('s') = }")

rps_elements.index('r') = 0
rps_elements.index('p') = 1
rps_elements.index('s') = 2


Since the table is also a list (of lists), we can retrieve one of its rows by index as follows.

In [4]:
>>> print(f"{rps_table[0] = }")  # Row 0 (the first row)

rps_table[0] = ['r', 'p', 'r']


And then, we can add another index reference to the row to retrieve a specific (column) element as shown below.

In [5]:
>>> print(f"{rps_table[0][2] = }")  # Row 0, Col 2 Element

rps_table[0][2] = 'r'


### Implementing a Binary Operator

Put all this together, we implement a *binary operator* as a **callable** Python class.

In [6]:
class BinaryOperator_v0:

    def __init__(self, elements, table):
        self._elements = elements
        self._table = table

    def __call__(self, elem1, elem2):
        row_index = self._elements.index(elem1)
        col_index = self._elements.index(elem2)
        return self._table[row_index][col_index]

Here are some examples of the RPS binary operator (version 0):

In [7]:
rps_bin_op_v0 = BinaryOperator_v0(rps_elements, rps_table)

print(f"{rps_bin_op_v0('r', 'p') = }")
print(f"{rps_bin_op_v0('p', 'p') = }")
print(f"{rps_bin_op_v0('s', 'p') = }")

rps_bin_op_v0('r', 'p') = 'p'
rps_bin_op_v0('p', 'p') = 'p'
rps_bin_op_v0('s', 'p') = 's'


In [17]:
>>> rps_table[0]

['r', 'p', 'r']

In [18]:
>>> [rps_elements.index(elem) for elem in rps_table[0]]

[0, 1, 0]

In [19]:
>>> [[rps_elements.index(elem) for elem in row] for row in rps_table]

[[0, 1, 0], [1, 1, 2], [0, 2, 2]]

In [20]:
def strings_to_indices(elements, table):
    """Convert a Cayley table of strings into one of integer indices."""
    return [[elements.index(elem) for elem in row] for row in table]

In [21]:
>>> strings_to_indices(rps_elements, rps_table)

[[0, 1, 0], [1, 1, 2], [0, 2, 2]]

## Cayley Table

In [15]:
import numpy as np

In [16]:
class CayleyTable:

    def __init__(self, arr):
        tmp = np.array(arr, dtype=int)
        nrows, ncols = tmp.shape
        if nrows == ncols:
            if (np.min(tmp) >= 0) and (np.max(tmp) < nrows):
                self._table = tmp
            else:
                raise Exception(f"All integers must be between 0 and {nrows - 1}, inclusive.")
        else:
            raise Exception(f"Input arrays must be square; this one is {nrows}x{ncols}.")

    def __repr__(self):
        return f"{self.__class__.__name__}({self._table.tolist()})"

    def __getitem__(self, tup):
        row, col = tup
        return self._table[row][col]

    def __hash__(self):
        return hash(tuple(self._table.tolist()))

    @property
    def table(self):
        return self._table

    def tolist(self):
        return self._table.tolist()

In [22]:
rps_ct = CayleyTable(strings_to_indices(rps_elements, rps_table))
rps_ct

CayleyTable([[0, 1, 0], [1, 1, 2], [0, 2, 2]])